<a href="https://colab.research.google.com/github/gayathry2612/experiments/blob/master/Data_Science_Test_Cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TESTING : Topics to cover

[How to do testing in Data Science? ](https://towardsdatascience.com/unit-testing-for-data-scientists-dc5e0cd397fb)

* As Data Science becomes more of a staple in large organizations, the need for proper testing of code slowly becomes more integrated into the skillset of a Data Scientist.

* Imagine you have been creating a pipeline for predicting customer churn in your organization. A few months after deploying your solution, there are new variables that might improve its performance.


* Unfortunately, after adding those variables, the code suddenly stops working! You are not familiar with the error message and you are having trouble finding your mistake. This is where testing, and specifically unit testing, comes in!

* Writing tests for specific modules improves the stability of your code and makes mistakes easier to spot. Especially when working on large projects, having proper tests is essentially a basic need.

# Why would you test your code?
Although this seems like a no-brainer, there are actually many reasons for testing your code:
* Prevent unexpected output
* Simplifies updating code
* Increases overall efficiency of developing code
* Helps to detect edge cases
* And most importantly prevents you from pushing any broken code into production!

And more reasons : 

Even for those who do write production code, it is important to write tests for at least the most important modules of their code.

# `What if you run a deep learning pipeline and fails after 3 hours on something you could have easily tested?`

Lets see how to do basic unittesting in Data Science pipelines. 